## Svenska Akademin  
WD egenskap [Property:P5325](https://www.wikidata.org/wiki/Property:P5325) 
* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenska%20Akademin.ipynb)

In [105]:
import urllib3, json
import pandas as pd   
from bs4 import BeautifulSoup
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import trange 

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?svaid WHERE {
?item wdt:P5325 ?svaid
}"""

http = urllib3.PoolManager()

# Query https://w.wiki/Vo5
def get_results(endpoint_url, query):
    user_agent = "user  salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def getChair (sakid):
    url = "http://www.svenskaakademien.se/svenska-akademien/ledamotsregister/" + sakid
    r = http.request('GET', url) 
    soup = BeautifulSoup(r.data, "html.parser")
    try:
        stol = soup.findAll("div", {"class": "field-name-field-stol"})[0].get_text()
        stol = stol.replace(u'Stol:\xa0', u'')
    except:
        stol =""
    try: 
        intrade = soup.findAll("div", {"class": "field-name-field-tog-intrade"})[0].get_text()
        intrade = intrade.replace(u'Inträde:\xa0',u'')
    except:
        intrade =""
    try: 
        avled = soup.findAll("div", {"class": "field-name-field-avled"})[0].get_text()
        avled = avled.replace(u'Utträde:\xa0',u'')
    except:
        avled =""
    return stol,intrade,avled


results = get_results(endpoint_url, query)
length = len (results["results"]["bindings"])
df = pd.DataFrame(columns=['WD', 'Chair', 'Start', 'End', 'SvenskaAkademin'])
for r in trange(0,length):
    result = results["results"]["bindings"][r]
    wd = result["item"]["value"].replace("http://www.wikidata.org/entity/","") 
    sakid = result["svaid"]["value"]
    chair, start, end = getChair(sakid) 
    df = df.append({'WD': wd, 'Chair': chair, 'Start': start, 'End': end, 'SvenskaAkademin': sakid }, ignore_index=True)
    

In [109]:
df.head(10)


,WD,Chair,Start,End,SvenskaAkademin
0,Q93137,8,1940-12-20,1974-07-11,lagerkvist-par
1,Q134641,11,1904-12-20,1931-04-08,karlfeldt-erik-axel
2,Q270468,15,1944-12-20,1949-01-07,wagner-elin
3,Q333297,13,1811-11-27,1847-08-14,franzen-frans-michael
4,Q346056,7,1852-05-18,1864-01-09,hagberg-carl-august
5,Q432129,16,1878-05-22,1895-09-21,rydberg-viktor
6,Q465251,14,1960-12-20,1966-03-27,josephson-ragnar
7,Q714281,3,1828-11-03,1847-12-25,brinkman-carl-gustaf-von
8,Q731379,11,1885-12-20,1904-06-11,odhner-clas-theodor
9,Q896571,3,1786-05-27,1794-02-15,celsius-olof


In [108]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 5 columns):
WD                 197 non-null object
Chair              197 non-null object
Start              197 non-null object
End                197 non-null object
SvenskaAkademin    197 non-null object
dtypes: object(5)
memory usage: 7.8+ KB


In [110]:
pd.set_option("display.max.rows", None) 

In [112]:
df["Start"] = pd.to_datetime(df["Start"])

In [122]:
df["End"] = pd.to_datetime(df["End"])  
df["Chair"] = df["Chair"].astype(int)


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 5 columns):
WD                 197 non-null object
Chair              197 non-null int64
Start              196 non-null datetime64[ns]
End                182 non-null datetime64[ns]
SvenskaAkademin    197 non-null object
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 7.8+ KB


In [124]:
df.sort_values(["Chair","Start"])

,WD,Chair,Start,End,SvenskaAkademin
55,Q491083,1,1786-04-05,1789-05-09,hopken-anders-johan-von
132,Q4358870,1,1789-12-11,1810-02-20,gyldenstolpe-nils-philip
43,Q1355965,1,1811-05-29,1839-06-30,wallin-johan-olof
16,Q1959091,1,1841-05-23,1881-03-21,fryxell-anders
61,Q2291097,1,1881-12-20,1901-07-31,forssell-hans
31,Q2070822,1,1902-06-20,1931-01-26,bildt-carl
80,Q2193692,1,1931-12-20,1945-10-02,wedberg-birger
181,Q2286404,1,1945-12-20,1968-11-30,ekeberg-birger
58,Q736172,1,1969-12-20,1976-12-13,petren-sture
13,Q1358507,1,1977-12-20,2008-11-29,rudholm-sten


In [ ]:
stolar = {1: }